# Portfolio Project on Biodiversity in National Parks
species_info.csv - contains data about different species and their conservation status. <br>
observations.csv - holds recorded sightings of different species at several national parks for the past 7 days.

In [68]:
import pandas as pd
species = pd.read_csv('species_info.csv')
print(species.head(5))

  category                scientific_name  \
0   Mammal  Clethrionomys gapperi gapperi   
1   Mammal                      Bos bison   
2   Mammal                     Bos taurus   
3   Mammal                     Ovis aries   
4   Mammal                 Cervus elaphus   

                                        common_names conservation_status  
0                           Gapper's Red-Backed Vole                 NaN  
1                              American Bison, Bison                 NaN  
2  Aurochs, Aurochs, Domestic Cattle (Feral), Dom...                 NaN  
3  Domestic Sheep, Mouflon, Red Sheep, Sheep (Feral)                 NaN  
4                                      Wapiti Or Elk                 NaN  


In [69]:
observations = pd.read_csv('observations.csv')
print(observations.head(5))

            scientific_name                            park_name  observations
0        Vicia benghalensis  Great Smoky Mountains National Park            68
1            Neovison vison  Great Smoky Mountains National Park            77
2         Prunus subcordata               Yosemite National Park           138
3      Abutilon theophrasti                  Bryce National Park            84
4  Githopsis specularioides  Great Smoky Mountains National Park            85


From observing the first 5 rows of each dataframe, we see that the dataframe _species_ has columns _category, scientific_name, common_names_ and _conservation_status_. The dataframe _observations_ has columns _scientific_name, park_name_ and _observations_. <br>
It makes sense to merge these tables on the common variable _scientific_name_ so that insights can be more easily visualised across the two tables. We will outer merge so that all data from both datasets is included.

In [70]:
df = pd.merge(left=species, right=observations, how='outer')
print(df.head(5))

         category scientific_name  \
0  Vascular Plant   Abies bifolia   
1  Vascular Plant   Abies bifolia   
2  Vascular Plant   Abies bifolia   
3  Vascular Plant   Abies bifolia   
4  Vascular Plant  Abies concolor   

                                        common_names conservation_status  \
0                          Rocky Mountain Alpine Fir                 NaN   
1                          Rocky Mountain Alpine Fir                 NaN   
2                          Rocky Mountain Alpine Fir                 NaN   
3                          Rocky Mountain Alpine Fir                 NaN   
4  Balsam Fir, Colorado Fir, Concolor Fir, Silver...                 NaN   

                             park_name  observations  
0                  Bryce National Park           109  
1            Yellowstone National Park           215  
2  Great Smoky Mountains National Park            72  
3               Yosemite National Park           136  
4  Great Smoky Mountains National Park       

The dataframes have been merged into a new dataframe called _df_. <br>
It is ordered lexiographically (according to the English alphabet) which will be useful if we need to fill missing data. <br>
Next we will explore the combined dataset using summary statistics.

In [71]:
df.nunique()

category                  7
scientific_name        5541
common_names           5504
conservation_status       4
park_name                 4
observations            304
dtype: int64

A count of unique values reveals that our dataframe describes observations at 4 different national parks of 5,541 distinct species.

In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25632 entries, 0 to 25631
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   category             25632 non-null  object
 1   scientific_name      25632 non-null  object
 2   common_names         25632 non-null  object
 3   conservation_status  880 non-null    object
 4   park_name            25632 non-null  object
 5   observations         25632 non-null  int64 
dtypes: int64(1), object(5)
memory usage: 700.9+ KB


We see that _df_ has 25,632 rows. All variables have datatype object which is expected for strings, except _observations_ which has datatype integer as expected for a counting variable. <br>
The count of non-null values for each variable reveals that _conservation_status_ has a significant number of missing values whereas all other columns are complete. <br>

In [73]:
df.conservation_status.value_counts()

conservation_status
Species of Concern    732
Endangered             80
Threatened             44
In Recovery            24
Name: count, dtype: int64

By getting more information about the counts of different values of _conservation_status_, we see that the status is either _Species of Concern, Endangered, Threatened,_ or _In Recovery_. <br>
We can use this context to infer that the _NaN_ values represent species that do not need to be conserved. <br>
Therefore this is structurally missing data. <br>
Alternatively it may be the case that _conservation_status_ is only provided once for each species in which case we'd need to forward fill _conservation_status_ by species. <br>
It could be also be the case that since different national parks are in different states, the species may be endangered in one state and not another. <br>
To rule out these possibilities we will print the overlap between species with a value for _conservation_status_ and species without a value for _conservation_status_.

In [74]:
conserved_species = df[df['conservation_status'].notna()]['scientific_name'].unique().tolist()
other = df[df['conservation_status'].isna()]['scientific_name'].unique().tolist()
both = set(conserved_species) & set(other)
print(both)

{'Oncorhynchus mykiss'}


We were expecting no overlap if our orginal theory was correct however we have found one species with at least two conservation statuses. Let's investigate further.

In [75]:
print(df[df['scientific_name']=='Oncorhynchus mykiss'])

      category      scientific_name   common_names conservation_status  \
15820     Fish  Oncorhynchus mykiss  Rainbow Trout                 NaN   
15821     Fish  Oncorhynchus mykiss  Rainbow Trout                 NaN   
15822     Fish  Oncorhynchus mykiss  Rainbow Trout                 NaN   
15823     Fish  Oncorhynchus mykiss  Rainbow Trout                 NaN   
15824     Fish  Oncorhynchus mykiss  Rainbow Trout                 NaN   
15825     Fish  Oncorhynchus mykiss  Rainbow Trout                 NaN   
15826     Fish  Oncorhynchus mykiss  Rainbow Trout                 NaN   
15827     Fish  Oncorhynchus mykiss  Rainbow Trout                 NaN   
15828     Fish  Oncorhynchus mykiss  Rainbow Trout          Threatened   
15829     Fish  Oncorhynchus mykiss  Rainbow Trout          Threatened   
15830     Fish  Oncorhynchus mykiss  Rainbow Trout          Threatened   
15831     Fish  Oncorhynchus mykiss  Rainbow Trout          Threatened   
15832     Fish  Oncorhynchus mykiss  R

Upon inspection, we see that the observation entries for the Rainbow Trout are duplicated, once with conservation status and once without. Perhaps the conservation status changed during the 7 days that observations were taken so were recorded with both values or perhaps one copy is a mistake. <br>
Since we do not reliably know its conservation status we could delete all rows relating to Oncorhynchus mykiss as it makes up only 16 of 25,632 rows.

In [76]:
print(str(round(16/25632 * 100,2))+'% of rows')

0.06% of rows


Alternatively we could keep the data that records _conservation_status_ as the worst case scenario so that rainbow trout are protected no matter what. This would mean deleting only the Rainbow Trout data where it has no recorded value for _conservation_status_. <br>
This time I've decided to delete all Rainbow Trout data as I am interested in which species are endangered and there is not reliable data on this subject for this species.

In [77]:
df = df[df['scientific_name']!='Oncorhynchus mykiss']

Let's also investigate other duplicates. We want to view all rows where species and observations are identical to a previous row.

In [78]:
duplicates = df[df.duplicated(subset=['scientific_name','park_name','observations'])]
print(len(duplicates))

2343
